# Potato Image Classification

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import models, layers 
print(tf.__version__)

In [2]:
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS=3
EPOCHS=10

## Preprocessing Data

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "potato_dataset",
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    shuffle=True  # Let TensorFlow handle shuffling
)

In [ ]:
for images, labels in dataset.take(1):
    print(images.shape, labels.shape)
    print(images[0].numpy())

In [ ]:
# class name
class_name = dataset.class_names
class_name

## Image Visualization

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
for images, labels in dataset.take(1):
    for i in range(10):
        ax = plt.subplot(4,4,i+1)
        
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_name[labels[i]])
        plt.axis("off")



## Splitting the Data

In [ ]:
dataset_size = len(dataset)
train_size = int(0.8*dataset_size)
val_size = int(0.1* dataset_size)
test_size = dataset_size-train_size-val_size

print(dataset_size)
print(train_size)
print(val_size)
print(test_size)


In [8]:
# Shuffle the dataset
dataset = dataset.shuffle(1000, seed=123)

# Split the dataset
train_dataset = dataset.take(train_size)
remaining = dataset.skip(train_size)
val_dataset = remaining.take(val_size)
test_dataset = remaining.skip(val_size)


## Caching and Prefetching

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)


print(f"Training dataset size: {len(train_dataset)} batches")
print(f"Validation dataset size: {len(val_dataset)} batches")
print(f"Testing dataset size: {len(test_dataset)} batches")

## Training Model

In [10]:
resize_and_rescal = tf.keras.Sequential(
    [
        tf.keras.layers.Resizing(IMAGE_SIZE,IMAGE_SIZE),
        tf.keras.layers.Rescaling(1./255)
        
    ]
)

In [11]:
data_augumentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomFlip(1./255),
        tf.keras.layers.RandomRotation(0.2),
    ]
) 

In [12]:
train_dataset = train_dataset.map(
    lambda x, y: (data_augumentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 3

model = models.Sequential([
    resize_and_rescal,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])


model.build(input_shape=input_shape)

In [ ]:
model.summary()

# Compiling The Model

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)
model.summary()

In [ ]:
history = model.fit(
    train_dataset,
    batch_size=BATCH_SIZE,
    validation_data=val_dataset,
    verbose=1,
    epochs=10,
)

In [ ]:

scores = model.evaluate(test_dataset)
scores

In [ ]:
print(history)
print(history.params)
print(history.history.keys())
print(type(history.history['loss']))
print(len(history.history['loss']))
print(history.history['loss'][:5])

In [19]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.plot(range(EPOCHS), acc, label='Training Accuracy')
plt.plot(range(EPOCHS), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(range(EPOCHS), loss, label='Training Loss')
plt.plot(range(EPOCHS), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Prediction

In [ ]:
import numpy as np
for images_batch, labels_batch in test_dataset.take(1):
    
    first_image = images_batch[0].numpy().astype('uint8')
    first_label = labels_batch[0].numpy()
    
    print("first image to predict")
    plt.imshow(first_image)
    print("actual label:",class_name[first_label])
    
    batch_prediction = model.predict(images_batch)
    print("predicted label:",class_name[np.argmax(batch_prediction[0])])

In [22]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = class_name[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [ ]:
plt.figure(figsize=(15, 15))
for images, labels in test_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        
        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = class_name[labels[i]] 
        
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
        
        plt.axis("off")

# Saving the Model

In [ ]:

import os
model_version=max([int(i) for i in os.listdir("./models/potatoes") + [0]])+1
os.makedirs(f"./models/potatoes/{model_version}", exist_ok=True)
model.save(f"./models/potatoes/{model_version}/potatoes_{model_version}.keras", save_format='keras')
# model.save("../potatoes.h5")

In [ ]:
# Save the model in SavedModel format (without specifying save_format)
saved_model_path = f"./saved_models/potatoes/{model_version}"
os.makedirs(saved_model_path, exist_ok=True)
model.export(saved_model_path)  # Saves in SavedModel format by default